In [ ]:
#import required packages
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#let us visualize the data.

import matplotlib.pyplot as plt
num = 20
plt.imshow(X_train[num])
X_train[num].shape

In [ ]:
# we can see the above output as 7 that will be present in the y_train[15]
print(y_train[num])

In [ ]:
# same for the test images it will be used for the validation
plt.imshow(X_test[num], cmap=plt.get_cmap('gray'))
print(y_test[num])
#so we have the input image data with the corresponding output data

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')
#reshaping every images to 28x28x1 width, height= 28, 28 then 1 represent 1 channel if it is 3 then it will a color image rgb

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
visualize_input(X_train[1].reshape(28,28), ax)
#now you can see the images with pixel ranging from 0 to 1 since we divided by 255.0

In [ ]:
#let us check
X_train[0]

In [ ]:
print('shape i.e the dimension of the images', X_train[0].shape)
# 28 rows 28 columns in the data in range of pixel value 0-255

In [ ]:
# it will be easily enhance the prediction through normalization by reducing the large values

X_train = X_train / 255
X_test = X_test / 255

In [ ]:
print(X_train[0])
# now it will be in range of 0-255

In [ ]:
#let us do one hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
#now 

print(y_train[15])
# 1 in the 7th index

In [ ]:
#total number of classes will be 10 i.e digit 0-9
num_classes = 10

In [ ]:
plt.imshow(X_train[0][:,:,0], cmap=plt.get_cmap('gray'))

In [ ]:
# defining CNN Model
def cnn():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# build the model
model = cnn()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
scores

In [ ]:
model.predict(X_test[:1])

In [ ]:
def expected_result(y_data):
  for i in range(10): 
    if(y_data[i]):
      print(i)
      break

In [ ]:
def predicted_result(pred_Xtrain, index):
  ans = pred_Xtrain[index].argsort()[-8:][::-1]  #sorting in descending
  print(ans[0])

In [ ]:
pred_Xtest = model.predict(X_test)

In [ ]:
index = 20
predicted_result(pred_Xtest, index)

In [ ]:
expected_result(y_test[index])

In [ ]:
model.save('/content/drive/My Drive/colab_data/digitrecognition.h5')
#save the model in the drive and download it for the deployment

In [ ]:
test1 = X_test[0]

In [ ]:
test1.shape

In [ ]:
X_test.shape

In [ ]:
test1 = test1.reshape(1, 28, 28, 1).astype('float32')
test1.shape

In [ ]:
model.predict(test1)

In [ ]:
type(test1)

In [ ]:
for i in range(0, 10):
  plt.imshow(X_train[100][:, :, 0])

In [ ]:
from tkinter import *
import tkinter.font as tkFont
import win32gui
from PIL import ImageGrab, Image
import numpy as np

from keras.models import load_model

'''
Let us deploy the model in the python application using tkinter
'''
class application(Frame):
    def __init__(self,master):
        super().__init__(master)
        self.fontStyle = tkFont.Font(family="Lucida Grande", size=20)
        self.master=master
        self.pack()
        self.createWidget()

    def createWidget(self):
        self.canvas = Canvas(self,width=284,height=284,bg='black')
        self.canvas.pack(expand=YES, fill=BOTH)
        self.canvas.bind('<B1-Motion>',self.activate_paint)

    def activate_paint(self, event):
        global lastx, lasty
        self.canvas.bind('<B1-Motion>', self.paint)
        lastx, lasty = event.x, event.y

    def paint(self,event):
        global lastx, lasty
        x, y = event.x, event.y
        self.canvas.create_line((lastx, lasty, x, y), width=12, fill='white')

        lastx, lasty = x, y

    def clearCanvas(self):
        self.canvas.delete("all")
        answer.configure(text='Answer Goes Here', font=self.fontStyle)
        self.canvas.bind('<B1-Motion>',self.activate_paint)

    def predicted_result(self, data):
        ans = data.argsort()[-8:][::-1]  #sorting in descending
        return ans


    def predictDigit(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        im = ImageGrab.grab(rect) # get image of the current location
        im.save('file.png')

        img = Image.open('file.png').convert('L')
        img = img.resize((28,28), Image.ANTIALIAS)
        img.save('resized.png')

        #after resizing the image data
        #convert to np array

        data = np.array(img)
        data = data/255.0 # for range b/w 0-1
        data = data.reshape(1, 28, 28, 1).astype('float32')

        #import the model
        model = load_model('./digitrecognition.h5')
        result = model.predict(data)
        ans = self.predicted_result(result)
        
        answer.configure(text='Predicted Digit: '+str(ans[0][-1]), font=self.fontStyle)

        
if __name__ == '__main__':
    root = Tk()
    root.geometry('500x400')
    
    
    app=application(root)
    # let us create button to clear the drawn items and predict the value

    clear = Button(root, text='Clear', command=app.clearCanvas)
    clear.pack()

    predict = Button(root, text='Predict', command=app.predictDigit)
    predict.pack()

    answer = Label(root, text="Answer Goes Here", font=app.fontStyle)
    answer.pack()

    root.title('Draw a Digit')
    root.mainloop()


    #let us run and see